In [1]:
import faiss
import torch
import wget
import os
import numpy as np
import pandas as pd

from omegaconf import OmegaConf
from pytorch_lightning import Trainer
from IPython.display import display
from tqdm import tqdm

from nemo.collections import nlp as nemo_nlp
from nemo.utils.exp_manager import exp_manager
import torch
torch.multiprocessing.set_sharing_strategy('file_system')

[NeMo W 2024-04-25 12:36:18 nemo_logging:349] /home/joheras/.local/lib/python3.10/site-packages/hydra/core/plugins.py:225: UserWarning: 
    	Error importing 'hydra_plugins.hydra_colorlog'.
    	Plugin is incompatible with this Hydra version or buggy.
    	Recommended to uninstall or upgrade plugin.
    		ImportError : cannot import name 'SearchPathPlugin' from 'hydra.plugins' (/home/joheras/.local/lib/python3.10/site-packages/hydra/plugins/__init__.py)
      warnings.warn(
    



===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please submit your error trace to: https://github.com/TimDettmers/bitsandbytes/issues


2024-04-25 12:36:19.864944: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
import numpy as np
import torch
from tqdm.auto import tqdm
from transformers import AutoTokenizer, AutoModel 

model = nemo_nlp.models.EntityLinkingModel.restore_from('model-SelfAlignmentPretrainingForMedicalEntityLinking-ClaraMeD-FineTuned/SelfAlignmentPretrainingForMedicalEntityLinking-ClaraMeD-FineTuned.nemo')

[NeMo W 2024-04-25 12:36:21 nlp_overrides:457] Apex was not found. Please see the NeMo README for installation instructions: https://github.com/NVIDIA/apex
    Megatron-based models require Apex to function correctly.
[NeMo W 2024-04-25 12:36:22 modelPT:161] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    data_file: ./DataLeo/train.txt
    max_seq_length: 128
    batch_size: 8
    shuffle: true
    num_workers: 2
    pin_memory: false
    drop_last: false
    
[NeMo W 2024-04-25 12:36:22 modelPT:168] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data() method and provide a valid configuration file to setup the validation data loader(s). 
    Validation config : 
    data_file: ./DataLeo/val.txt
    max_seq_length: 128
    batch_size: 8
    shuffle: false
    num_workers: 2


[NeMo I 2024-04-25 12:36:26 nlp_overrides:755] Model EntityLinkingModel was successfully restored from /home/joheras/CLARA-Med/Normalisation/model-SelfAlignmentPretrainingForMedicalEntityLinking-ClaraMeD-FineTuned/SelfAlignmentPretrainingForMedicalEntityLinking-ClaraMeD-FineTuned.nemo.


In [3]:
import pandas as pd

In [15]:
df = pd.read_csv('DataLeo/umls_spa_cui_term_test.txt',sep='\|\|',header=None)

[NeMo W 2024-04-25 12:48:17 nemo_logging:349] /tmp/ipykernel_2438651/1043993813.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
      df = pd.read_csv('DataLeo/umls_spa_cui_term_test.txt',sep='\|\|',header=None)
    


In [27]:
df.columns=['Term','Code']

In [5]:
test_kb = OmegaConf.create({
    "data_file": os.path.join('DataLeo/umls_spa_cui_term_test.txt'),
    "max_seq_length": 128,
    "batch_size": 128,
    "shuffle": False,
})

In [6]:
test_kb_dataloader = model.setup_dataloader(test_kb, is_index_data=True)

[NeMo I 2024-04-25 12:36:26 entity_linking_dataset:78] Loaded dataset with 1247174 examples


In [7]:
# Helper function to get data embeddings
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

def get_embeddings(model, dataloader):
    embeddings, cids = [], []

    with torch.no_grad():
        for batch in tqdm(dataloader):
            input_ids, token_type_ids, attention_mask, batch_cids = batch
            batch_embeddings = model.forward(input_ids=input_ids.to(device), 
                                             token_type_ids=token_type_ids.to(device), 
                                             attention_mask=attention_mask.to(device))

            # Accumulate index embeddings and their corresponding IDs
            embeddings.extend(batch_embeddings.cpu().detach().numpy())
            cids.extend(batch_cids)
            
    return embeddings, cids

In [8]:
embeddings,_ = get_embeddings(model,test_kb_dataloader)
embeddings = np.array(embeddings)

  0%|          | 0/9744 [00:00<?, ?it/s]

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

In [12]:
quantizer = faiss.IndexFlatL2(768)
index = faiss.IndexIVFFlat(quantizer, 768, 50)
# index = faiss.index_cpu_to_all_gpus(index)
index.train(embeddings)

In [13]:
for i in tqdm(range(0, embeddings.shape[0], 128)):
    index.add(embeddings[i:i+128])

  0%|          | 0/9744 [00:00<?, ?it/s]

In [14]:
faiss.write_index(index,'dataLeo_sap_bert_finetuned_index')

In [16]:
k=5

In [17]:
test_data = pd.read_csv('DataLeo/test_norm_ref.tsv',sep='\t',header=None)

In [20]:
test_data.loc[:,4][0:10]

0                                nicotina
1                                    iPTH
2                          fosfato sérico
3                                     HDL
4                         trombocitopenia
5                            hemodiálisis
6                            nicotinamida
7                         hiperfosfatemia
8                            hemodiálisis
9    evaluaron la eficacia y la seguridad
Name: 4, dtype: object

In [21]:
all_names = list(test_data.loc[:,4])
bs = 128 # batch size during inference
all_embs = []
for i in tqdm(np.arange(0, len(all_names), bs)):
    
    model_input =  model.tokenizer(all_names[i:i+bs],
                                   add_special_tokens = True,
                                   padding = True,
                                   truncation = True,
                                   max_length = 512,
                                   return_token_type_ids = True,
                                   return_attention_mask = True)
    
    # Pass tokenized input into model
    query_emb =  model.forward(input_ids=torch.LongTensor(model_input["input_ids"]).to(device),
                               token_type_ids=torch.LongTensor(model_input["token_type_ids"]).to(device),
                               attention_mask=torch.LongTensor(model_input["attention_mask"]).to(device))
    
    
    
    all_embs.append(query_emb.cpu().detach().numpy())

all_embs = np.concatenate(all_embs, axis=0)

  0%|          | 0/88 [00:00<?, ?it/s]

In [22]:
k = 5

In [24]:
D, I = index.search(all_embs, k) 

In [25]:
I

array([[ 791489,  791490,  791460,   63875,  850047],
       [ 989136,  607973,  988966, 1156852, 1168167],
       [ 513638,  514595,  514229,  513907,  871321],
       ...,
       [1089286,  422003,  422001,  422004,  422009],
       [ 860516,  860517,  822751,  341828,  860515],
       [1030137, 1030135, 1030136, 1030134, 1091591]])

In [28]:
codes = df['Code'].iloc[I[0]].values

In [29]:
codes

array(['C0028040', 'C0028050', 'C0028027', 'C0028053', 'C1253423'],
      dtype=object)

In [30]:
test_data.loc[:,5][0] in codes

True

In [31]:
aciertosTop1 = 0
aciertosTop5 = 0

for i,x in tqdm(enumerate(I)):
    codes = df['Code'].iloc[x].values
    code = test_data.loc[:,5][i]
    if (code==codes[0]):
        aciertosTop1 += 1
    if (code in codes):
        aciertosTop5 += 1

0it [00:00, ?it/s]

In [32]:
print("Accuracy Top 1 %f" % (aciertosTop1/len(I)))
print("Accuracy Top 5 %f" % (aciertosTop5/len(I)))

Accuracy Top 1 0.657874
Accuracy Top 5 0.791469


In [35]:
test_data_uncert = pd.read_csv('DataLeo/test_norm_ref_with_uncert.tsv',sep='\t',header=None)

In [36]:
all_names = list(test_data_uncert.loc[:,4])
bs = 128 # batch size during inference
all_embs = []
for i in tqdm(np.arange(0, len(all_names), bs)):
    
    model_input =  model.tokenizer(all_names[i:i+bs],
                                   add_special_tokens = True,
                                   padding = True,
                                   truncation = True,
                                   max_length = 512,
                                   return_token_type_ids = True,
                                   return_attention_mask = True)
    
    # Pass tokenized input into model
    query_emb =  model.forward(input_ids=torch.LongTensor(model_input["input_ids"]).to(device),
                               token_type_ids=torch.LongTensor(model_input["token_type_ids"]).to(device),
                               attention_mask=torch.LongTensor(model_input["attention_mask"]).to(device))
    
    
    
    all_embs.append(query_emb.cpu().detach().numpy())

all_embs = np.concatenate(all_embs, axis=0)

  0%|          | 0/91 [00:00<?, ?it/s]

In [37]:
k=5

In [38]:
D, I = index.search(all_embs, k) 

In [39]:
aciertosTop1 = 0
aciertosTop5 = 0

for i,x in tqdm(enumerate(I)):
    codes = df['Code'].iloc[x].values
    code = test_data_uncert.loc[:,5][i]
    if (code==codes[0]):
        aciertosTop1 += 1
    if (code in codes):
        aciertosTop5 += 1

0it [00:00, ?it/s]

In [40]:
print("Accuracy Top 1 %f" % (aciertosTop1/len(I)))
print("Accuracy Top 5 %f" % (aciertosTop5/len(I)))

Accuracy Top 1 0.635103
Accuracy Top 5 0.770636
